# ECG Feature Extraction Notebook

## Libraries

In [1]:
import pandas as pd
import os
import numpy as np
import glob
import antropy as ant


## Extract Dictionary Linking Subject IDs and R Peaks

Given: eyes open/closed (approx. 30 seconds), and ecg baseline (5 minute segments)

In [34]:
def make_ecg_dict(folder_path ='/Users/{User}/Desktop/codebase_ecg_mtbi_feature extraction/Example Data', segment_type = ['eyes_open_','eyes_closed_','ecg_baseline']):
    """
    Desc: Dictionary where keys --> subject id and vals --> dictionary where keys are "open" or "closed" or "five minutes" and vals are a list of rr intervals

    Inputs: 
    folder_path  -->  directory of folders containing ecg data csvs, 
    segment_type --> open, closed, or five minutes

    Outputs: description
    """
    ecg_data = {}
    rpeak_arrays = []

    folder_path = '/Users/{User}/Desktop/codebase_ecg_mtbi_feature extraction/Example Data/'
    segment_type = ['eyes_open_','eyes_closed_','ecg_baseline']

    # CHANGE THIS WHEN CHANGING SEGMENT TYPES
    seg_type = segment_type[2]

    csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
    for csv_file in csv_files:
        csv_file = csv_file.replace("\\",'/')
        rpeak_array = pd.read_csv(csv_file, usecols = ['RR-interval (ms)']).values
        for seg_types in segment_type:
            if seg_type in csv_file:
                ecg_data = {seg_type: rpeak_array}

    return ecg_data

## Feature Extraction Functions

Mean RR, Mean HR, Max HR, Min HR, STD HR, HRV, RMSSD, PNN50, 75th Percentile, 25th Percentile, IQR

In [36]:
def calc_mean_rr(rr_array):
    return np.nanmean(rr_array)    

In [37]:
def calc_mean_hr(rr_array):
    
    return np.nanmean(60000/rr_array)

In [38]:
def calc_max_hr(rr_array):
    return np.nanmax(60000/rr_array)

In [39]:
def calc_min_hr(rr_array):
    return np.nanmin(60000/rr_array)

In [40]:
def calc_std_hr(rr_array):
    return np.nanstd(60000/rr_array)

In [41]:
def calc_hrv(rr_array):
    return np.nanstd(rr_array)

In [42]:
def calc_rmssd(rr_array):
    rr_array = rr_array[~np.isnan(rr_array)]
    return np.sqrt(np.mean(np.square(np.diff(rr_array))))

In [43]:
def calc_pnn50(rr_array):
    rr_array = rr_array[~np.isnan(rr_array)]
    ibi_differences = np.abs(np.diff(rr_array))
    ibi_differences[ibi_differences < 50] = 0
    pnn50 = len(np.nonzero(ibi_differences)[0]) / len(ibi_differences)
    return pnn50

In [44]:
def calc_75_perc(rr_array):
    return np.nanpercentile(rr_array, 75)

def calc_25_perc(rr_array):
    return np.nanpercentile(rr_array, 25)

def calc_iqr(rr_array):
    return calc_75_perc(rr_array) - calc_25_perc(rr_array)


In [45]:
def calc_perm_ent(rr_array):
    return ant.perm_entropy(rr_array, normalize=True)


def calc_spec_ent(rr_array):
    return ant.spectral_entropy(rr_array, sf=500, method='welch', normalize=True)

def calc_svd_ent(rr_array):
    return ant.svd_entropy(rr_array, normalize=True)

def calc_approx_ent(rr_array):
    return ant.app_entropy(rr_array)

def calc_sample_ent(rr_array):
    return ant.sample_entropy(rr_array)

def calc_hjorth_mob(rr_array):
    return ant.hjorth_params(rr_array)[0]

def calc_hjorth_comp(rr_array):
    return ant.hjorth_params(rr_array)[1]

def calc_zero_cross(rr_array):
    return ant.num_zerocross(rr_array)

# Extract all Features!

In [47]:
def get_ecg_features(ecg_data, feature_names = ["mean RR", "mean HR", "max HR", "min HR", "stdev HR", "hrv", "rmssd", "pnn50", "75th percentile", "25th percentile", "iqr",
                                                              "permutation entropy", "spectral entropy", "svd entropy", "approximate entropy", "sample entropy", "hjorth mobility", "hjorth complexity"], \
                     segment_type = 'ecg_baseline' ):
    """
    Desc: makes dictionary where keys are segment types and values are numpy arrays of subject ids x features
    Inputs: 
    ecg_data -> Dictionary where keys --> subject id and vals --> dictionary where keys are "open" or "closed" or "five minutes" and vals are a list of rr intervals
    Outputs: 
    ids_x_feat_dict_array -> dictionary where keys are segment types and values are numpy arrays of subject ids x features
    """
    print("Calculating ECG features")

    feature_names = ["mean RR", "mean HR", "max HR", "min HR", "stdev HR", "hrv", "rmssd", "pnn50", "75th percentile", "25th percentile", "iqr",
                    "permutation entropy", "spectral entropy", "svd entropy", "approximate entropy", "sample entropy", "hjorth mobility", "hjorth complexity"] 

    ids_x_feat = []
    id_features = []

    ibi = ecg_data[segment_type]
    ibi_array = ibi[~np.isnan(ibi)]
    for feature_ind,feature_type in enumerate(feature_names):
        if feature_type == "mean RR":
            id_features.append(calc_mean_rr(ibi))
        elif feature_type == "mean HR":
            id_features.append(calc_mean_hr(ibi))
        elif feature_type == "max HR":
            id_features.append(calc_max_hr(ibi))
        elif feature_type == "min HR":
            id_features.append(calc_min_hr(ibi))
        elif feature_type == "stdev HR":
            id_features.append(calc_std_hr(ibi))
        elif feature_type == "hrv":
            id_features.append(calc_hrv(ibi))
        elif feature_type == "rmssd":
            id_features.append(calc_rmssd(ibi))
        elif feature_type == "pnn50":
            id_features.append(calc_pnn50(ibi))
        elif feature_type == "75th percentile":
            id_features.append(calc_75_perc(ibi))
        elif feature_type == "25th percentile":
            id_features.append(calc_25_perc(ibi))
        elif feature_type == "iqr":
            id_features.append(calc_iqr(ibi))
        elif feature_type == "permutation entropy":
            id_features.append(calc_perm_ent(ibi_array))
        elif feature_type == "spectral entropy":
            id_features.append(calc_spec_ent(ibi_array))
        elif feature_type == "svd entropy":
            id_features.append(calc_svd_ent(ibi_array))
        elif feature_type == "approximate entropy":
            id_features.append(calc_approx_ent(ibi_array))
        elif feature_type == "sample entropy":
            id_features.append(calc_sample_ent(ibi_array))
        elif feature_type == "hjorth mobility":
            id_features.append(calc_hjorth_mob(ibi_array))
        elif feature_type == "hjorth complexity":
            id_features.append(calc_hjorth_comp(ibi_array))

            # Add to the list of list of features!
            ids_x_feat.append(id_features)
            id_features = []

        else:
            raise ValueError(f"{feature_type} not defined") 
                
    # Make 3 dataframes of subject ids x feature names based on segment type (the keys)
    ids_x_feat_dict_array = {segment_type: ids_x_feat}
    df_list = []
    df_features = pd.DataFrame(data=ids_x_feat, index=segment_type, columns=feature_names)
    df_list.append(df_features)
    df_features.to_csv(f"/Users/{User}/Desktop/codebase_ecg_mtbi_feature extraction/Example Data/{segment_type}.csv")
    return ids_x_feat_dict_array

In [48]:
get_ecg_features(make_ecg_dict())


Calculating ECG features


c:\Users\joshu\miniconda3\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 212, using nperseg = 212
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\joshu\miniconda3\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 243, using nperseg = 243
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\joshu\miniconda3\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 241, using nperseg = 241
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\joshu\miniconda3\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 208, using nperseg = 208
  warnings.warn('nperseg = {0:d} is greater than input length '
c:\Users\joshu\miniconda3\lib\site-packages\scipy\signal\_spectral_py.py:2014: UserWarning: 

{'ecg_baseline': [[1133.1666666666667,
   53.300800140711885,
   72.63922518159806,
   44.642857142857146,
   4.578522134747427,
   87.76122622367369,
   115.90621618606286,
   0.7718631178707225,
   1196.0,
   1089.5,
   106.5,
   0.992989254326706,
   0.7640387402049329,
   0.301493950899511,
   1.0179559445710264,
   1.8128128784258872,
   1.3206586644679548,
   1.2644229825934463],
  [677.4487471526196,
   88.60340784718542,
   93.75,
   84.26966292134831,
   1.7899105043827945,
   13.562526828201333,
   5.853617700418127,
   0.0,
   688.0,
   668.0,
   20.0,
   0.9526096915306702,
   0.6648314091531046,
   0.0461389883236212,
   1.1322214077821036,
   1.1870563306592352,
   0.43158916288393484,
   3.3225251988233913],
  [658.7312775330397,
   91.28921745591927,
   104.8951048951049,
   80.21390374331551,
   4.392689933577859,
   30.832402425647476,
   18.915170220200537,
   0.033112582781456956,
   676.0,
   644.0,
   32.0,
   0.9312211848626287,
   0.6965361017434137,
   0.121821